In [1]:
import requests
import re
import operator
from time import sleep
from bs4 import BeautifulSoup
from functools import reduce
from datetime import datetime

domain = "https://www.nikkan.co.jp"
startpath = "/"

In [2]:
basesoup = BeautifulSoup(requests.get(domain + startpath).text, "lxml")

In [3]:
links = []
for a in basesoup.find_all("a", href=re.compile("^/articles/view/[0-9]+")):
    links.append(domain + a.get("href"))
links = list(set(links))

In [4]:
texts = []
count = 0

for link in links:
    sleep(2)
    res = requests.get(link)
    res.encoding = res.apparent_encoding
    soup = BeautifulSoup(res.text, "lxml")

    title = reduce(operator.add ,[x.text.strip() for x in soup.find_all("div", class_="ttl")], "")
    main = reduce(operator.add ,[x.text.strip() for x in soup.find_all("p", style="text-indent:1em;")], "")
    text = (title + " " + main).replace("\u3000"," ")
    texts.append(text)
    
    count = count + 1
    print (datetime.now().isoformat()+":("+str(count)+"/"+str(len(links))+")")

2017-11-16T17:28:12.631621:(1/220)
2017-11-16T17:28:15.671669:(2/220)
2017-11-16T17:28:18.598072:(3/220)
2017-11-16T17:28:21.647107:(4/220)
2017-11-16T17:28:24.534836:(5/220)
2017-11-16T17:28:27.369914:(6/220)
2017-11-16T17:28:30.460937:(7/220)
2017-11-16T17:28:33.292681:(8/220)
2017-11-16T17:28:36.290981:(9/220)
2017-11-16T17:28:39.162009:(10/220)
2017-11-16T17:28:42.124693:(11/220)
2017-11-16T17:28:45.071747:(12/220)
2017-11-16T17:28:47.936547:(13/220)
2017-11-16T17:28:50.751683:(14/220)
2017-11-16T17:28:53.621995:(15/220)
2017-11-16T17:28:56.675449:(16/220)
2017-11-16T17:28:59.501685:(17/220)
2017-11-16T17:29:02.644106:(18/220)


KeyboardInterrupt: 

In [14]:
import sqlite3
import hashlib

dbname = "text.db"
dbcon = sqlite3.connect(dbname)
dbcur = dbcon.cursor()

for text in texts:
    insert = "INSERT INTO rawtext(id, source, time, rawtext) VALUES(?, ?, ?, ?)"

    id = hashlib.md5(text.encode("utf-8")).hexdigest()
    source = "日刊工業新聞"
    time = datetime.now().isoformat()
    
    args = (id, source, time, text)
    
    try:
        dbcur.execute(insert, args)
    except sqlite3.Error as e:
        print('sqlite3:', e.args[0])
    
dbcon.commit()
dbcon.close()
print (datetime.now().isoformat()+":db written")